In [12]:
from pathlib import Path

import pandas as pd
import numpy as np
from fedot.api.main import Pipeline, Fedot
from fedot.api.builder import FedotBuilder
from fedot.core.data.data import InputData
from fedot.core.data.data_split import train_test_data_setup

In [3]:
data_root = '../data/boston-housing'

## 1. Загрузка данных

In [4]:
data = InputData.from_csv(
    Path(data_root, 'train.csv'),
    task='regression',
    index_col='ID',
    target_columns='medv'
)
train, test = train_test_data_setup(data, shuffle=True)

In [15]:
pd.DataFrame(np.concatenate([data.features, data.target], axis=1), index=data.idx, columns=[*data.features_names, 'medv'])

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
2,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
4,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
7,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,0.17783,0.0,9.69,0.0,0.585,5.569,73.5,2.3999,6.0,391.0,19.2,395.77,15.10,17.5
502,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
503,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
504,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9


## 2. Создание и обучение модели

In [47]:
model = (FedotBuilder(problem='regression')
         .setup_composition(timeout=0.01, preset='fast_train', with_tuning=True)
         .setup_pipeline_evaluation(metric="mse", cv_folds=3)
         .setup_data_preprocessing(use_auto_preprocessing=True)
         .build())
pipeline = model.fit(features=train)

2025-02-28 04:32:25,692 - ApiDataProcessor - Preprocessing data
2025-02-28 04:32:25,694 - ApiDataProcessor - Train Data (Original) Memory Usage: 27.14 KB Data Shapes: ((266, 13), (266, 1))
2025-02-28 04:32:25,703 - ApiDataProcessor - Train Data (Processed) Memory Usage: 43.77 KB Data Shape: ((266, 21), (266, 1))
2025-02-28 04:32:25,705 - ApiDataProcessor - Data preprocessing runtime = 0:00:00.014127
2025-02-28 04:32:26,440 - ApiComposer - Initial pipeline was fitted in 2.2 sec.
2025-02-28 04:32:26,448 - ApiComposer - AutoML configured. Parameters tuning: True. Time limit: 0.01 min. Set of candidate models: ['adareg', 'dtreg', 'knnreg', 'lasso', 'linear', 'normalization', 'pca', 'ransac_lin_reg', 'rfr', 'ridge', 'scaling', 'sgdr'].
2025-02-28 04:32:26,479 - ApiComposer - Timeout is too small for composing and is skipped because fit_time is 2.178708 sec.
2025-02-28 04:32:26,487 - ApiComposer - Time for pipeline composing was 0:00:00.
The remaining 0 seconds are not enough to tune the hyp

In [48]:
pipeline.graph_description

{'depth': 2, 'length': 2, 'nodes': [rfr, scaling]}

## 3. Подсчёт метрик

In [49]:
model.predict(test)
model.get_metrics()

2025-02-28 04:32:32,590 - ApiDataProcessor - Preprocessing data
2025-02-28 04:32:32,591 - ApiDataProcessor - Test Data (Original) Memory Usage: 56.0 B Data Shapes: ((67, 13), (67, 1))
2025-02-28 04:32:32,597 - ApiDataProcessor - Test Data (Processed) Memory Usage: 56.0 B Data Shape: ((67, 21), (67, 1))
2025-02-28 04:32:32,599 - ApiDataProcessor - Data preprocessing runtime = 0:00:00.008510


{'mse': 7.932, 'mape': 0.113}

## 4. Сохранение и загрузка модели

In [50]:
pipeline.save('runs/boston-housing', create_subdir=False)

('{\n    "total_pipeline_operations": [\n        "scaling",\n        "rfr"\n    ],\n    "depth": 2,\n    "nodes": [\n        {\n            "operation_id": 1,\n            "operation_type": "scaling",\n            "operation_name": "ScalingImplementation",\n            "custom_params": {},\n            "params": {},\n            "nodes_from": [],\n            "fitted_operation_path": [\n                "fitted_operations",\n                "operation_1.pkl"\n            ],\n            "rating": null\n        },\n        {\n            "operation_id": 0,\n            "operation_type": "rfr",\n            "operation_name": "RandomForestRegressor",\n            "custom_params": {\n                "n_jobs": 16\n            },\n            "params": {\n                "n_jobs": 16\n            },\n            "nodes_from": [\n                1\n            ],\n            "fitted_operation_path": [\n                "fitted_operations",\n                "operation_0.pkl"\n            ],\n  

In [ ]:
FedotBuilder

In [5]:
load_pipeline = Pipeline().load('runs/boston-housing')

2025-02-28 15:20:12,426 - Topological features operation requires extra dependencies for time series forecasting, which are not installed. It can infuence the performance. Please install it by 'pip install fedot[extra]'


In [7]:
test_data = pd.read_csv(Path(data_root, 'test.csv'))

In [19]:
load_model = Fedot('regression')
load_model.load('runs/boston-housing')

In [21]:
load_model.predict(test_data)

array([18.892, 18.507, 16.899, 16.425, 15.817, 13.765, 13.711, 13.639,
       13.639, 13.697, 13.697, 13.697, 13.693, 13.693, 13.693, 13.693,
       13.693, 13.732, 13.693, 13.86 , 13.872, 13.872, 13.731, 13.711,
       13.711, 13.711, 13.872, 13.711, 13.872, 13.711, 13.711, 13.711,
       13.711, 13.711, 13.711, 13.711, 13.711, 13.711, 13.711, 13.711,
       13.711, 13.711, 13.711, 13.711, 13.711, 13.711, 13.711, 13.711,
       13.711, 10.034, 10.034, 13.711, 10.034, 13.711, 13.711, 13.711,
       13.872, 13.892, 13.872, 13.872, 13.872, 13.872, 13.711, 10.034,
       10.034, 10.034, 13.711, 13.711, 10.034, 10.034, 10.034, 13.711,
       13.711, 13.711, 13.872, 13.872, 13.872, 13.872, 13.872, 13.872,
       13.872, 13.872, 13.872, 13.878, 13.878, 13.878, 13.878, 10.233,
       13.878, 13.872, 10.233, 10.233, 13.872, 13.878, 13.872, 13.872,
       13.872, 13.711, 13.711, 13.872, 13.872, 13.872, 13.872, 13.711,
       13.711, 13.711, 13.711, 13.711, 13.711, 13.711, 13.711, 13.872,
      

In [11]:
load_pipeline.predict(test_data)

TypeError: 'numpy.ndarray' object is not callable

In [4]:
test_data = pd.read_csv(Path(data_root, 'test.csv'))
InputData.from_dataframe(
    test_data,
    pd.DataFrame()
)

InputData(idx=array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177